# Direktna metoda izračuna astronomskega položaja

Sledijo komentarji

In [1]:
import os, sys

# add custom modules and astro data path 
pp = '../nav_tools/'
sys.path.append(pp)

In [2]:
import math as mat
import numpy as np
import matplotlib.pyplot as mpl
mpl.rcParams['text.usetex'] = True
mpl.rcParams.update({'font.size': 7})

import celestialdata as cdata
import navigationalstars as ns
import navtools as nt

In [3]:
#

def deg2rad(dd, mm=0):
    
    return (dd + mm/60)/180*mat.pi

def rad2deg(x):
    
    neg = False
    if x < 0:
        neg = True
    xx = np.abs(x)
    dd = mat.floor(xx/mat.pi*180)
    mm = (xx/mat.pi*180 - dd)*60
    
    if neg:
        return '-{:02d} {:08.5f}'.format(dd,mm)
    else:
        return '{:02d} {:08.5f}'.format(dd,mm)

In [4]:
# Returns Geographic Position (GP) of the star based on DEC and GHA
# input: declination, GHA
# output: fi, lambda

def getZFL(h, dec, gha):
    
    z = mat.pi/2 - h
    fi = dec
    if gha > mat.pi:
        la = 2*mat.pi - gha
    else:
        la = -gha
    
    return [z,fi,la]

In [5]:
# Returns great circle (GC) distance between two positions
# - input: fi1, fi2, dl (delta lambda)
# - output: great circle distance in radians

def getDist(fi1,fi2,dl):
    
    psi1 = mat.pi/2 - fi1
    psi2 = mat.pi/2 - fi2
    
    cosD = mat.cos(psi1)*mat.cos(psi2) + mat.sin(psi1)*mat.sin(psi2)*mat.cos(dl)
    
    return mat.acos(cosD)

In [6]:
#

def getPi1(z1,z2,D):
    
    cosPi1 = (mat.cos(z2) - mat.cos(z1)*mat.cos(D)) / (mat.sin(z1)*mat.sin(D))
    
    return mat.acos(cosPi1)

In [7]:
#

def getW1(fi1,fi2,D):
    
    psi1 = mat.pi/2 - fi1
    psi2 = mat.pi/2 - fi2
    
    cosW1 = (mat.cos(psi2) - mat.cos(psi1)*mat.cos(D)) / (mat.sin(psi1)*mat.sin(D))
    
    return mat.acos(cosW1)

In [8]:
#

def getPsiX(z1, fi1, Wx):
    
    psi1 = mat.pi/2 - fi1
    
    cosPsiX = mat.cos(psi1)*mat.cos(z1) + mat.sin(psi1)*mat.sin(z1)*mat.cos(Wx)
    
    return mat.acos(cosPsiX)

In [9]:
#

def getDLx(z1, fi1, fix):
    
    psi1 = mat.pi/2 - fi1
    psix = mat.pi/2 - fix
    
    cosDL = (mat.cos(z1) - mat.cos(psi1)*mat.cos(psix)) / (mat.sin(psi1)*mat.sin(psix))
    
    return mat.acos(cosDL)

In [10]:
#

def calculatePosition(z1,fi1,la1,z2,fi2,la2):
    
    # delta fi and delta lambda
    dfi = fi2 - fi1
    dla = la2 - la1
    if dla > mat.pi:
        dla = -(2*mat.pi - dla)
    elif dla < -mat.pi:
        dla = 2*mat.pi + dla

    #print('dla={:}'.format(rad2deg(dla)))

    D = getDist(fi1,fi2, dla)
    Pi1 = getPi1(z1,z2,D)
    W1 = getW1(fi1,fi2,D)
    alpha1 = W1 - Pi1
    print('D={:}, W1={:}, Pi1={:}, a1={:}'.format(rad2deg(D), rad2deg(W1), rad2deg(Pi1), rad2deg(alpha1)))

    psiX = getPsiX(z1,fi1,alpha1)
    fiX = mat.pi/2 - psiX
    dlX = getDLx(z1,fi1,fiX)
    print('psiX={:}, dlX={:}'.format(rad2deg(psiX), rad2deg(dlX)))
    
    if dla < 0:
        laX = la1 - np.abs(dlX)
    else:
        laX = la1 + np.abs(dlX)
    
    return [fiX, laX]

In [11]:
# observer position (what should we get)!!
# fi_check = deg2rad(46,0)
# la_check = deg2rad(13,30)

# star 1
h1 = deg2rad(31,02.2)
dec1 = deg2rad(7,24.7)
gha1 = deg2rad(294,21.7)
[z1,fi1,la1] = getZFL(h1,dec1,gha1)
print('1: z={:}, fi={:}, la={:}'.format(rad2deg(z1),rad2deg(fi1),rad2deg(la1)))

# star 2
h2 = deg2rad(67,13.9)
dec2 = deg2rad(23,34.3)
gha2 = deg2rad(351,21.0)
[z2,fi2,la2] = getZFL(h2,dec2,gha2)
print('2: z={:}, fi={:}, la={:}'.format(rad2deg(z2),rad2deg(fi2),rad2deg(la2)))

[fiX, laX] = calculatePosition(z1,fi1,la1,z2,fi2,la2)

print()
print('-----------------------------------------')
print('Calculated position')
print('fi={:}, la={:}'.format(rad2deg(fiX), rad2deg(laX)))
#print()
#print('Error in calculated position')
#print('fi={:.2f} Nm, la={:.2f} Nm'.format(np.abs(fi-fiX)/mat.pi*180*60, np.abs(la-laX)/mat.pi*180*60))

1: z=58 57.80000, fi=07 24.70000, la=65 38.30000
2: z=22 46.10000, fi=23 34.30000, la=08 39.00000
D=56 51.27987, W1=66 37.73072, Pi1=26 49.96233, a1=39 47.76839
psiX=43 59.99865, dlX=52 08.27296

-----------------------------------------
Calculated position
fi=46 00.00135, la=13 30.02704
